In [34]:
#!pip install timm

In [35]:
import pandas as pd
import timm
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
#from efficientnet_pytorch import EfficientNet
import numpy as np

from PIL import Image
from PIL import ImageFile

In [36]:
import pytorch_lightning
pytorch_lightning.seed_everything(42)

42

In [37]:
base_dir = r'C:\Users\Kaggle\Leaf_Classification\input'

In [38]:
### read csv and label mapping

labels = pd.read_csv(os.path.join(base_dir,'train.csv'))

with open(f'{base_dir}/label_num_to_disease_map.json') as f:
    label_mapper = json.load(f)

### Dataset class

In [39]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names: List,
        transforms: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """

        self.mode = mode
        self.transforms = transforms
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.img_path + self.image_names[idx]
        image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64')}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)

### Augs

In [40]:
sz = 576
sz_efffnetb4 = 600
deit_size = 384

In [41]:
# TTA0_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             ToTensorV2()],
#             p=1.)


# TTA1_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.HorizontalFlip(p=1),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             ToTensorV2()],
#             p=1.)


# TTA2_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.VerticalFlip(p=1),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             ToTensorV2()],
#             p=1.)


# TTA2_5_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.Transpose(p=1.),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             ToTensorV2()],
#             p=1.)

# TTA3_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.HueSaturationValue(
#                 hue_shift_limit=0.2, 
#                 sat_shift_limit=0.2, 
#                 val_shift_limit=0.2, 
#                 p=1
#             ),
#            albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             ToTensorV2()],
#             p=1.)


# TTA4_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.RandomBrightnessContrast(
#                 brightness_limit=(-0.1,0.1), 
#                 contrast_limit=(-0.1, 0.1), 
#                 p=1.
#             ),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             ToTensorV2()],
#             p=1.)


# TTA5_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             albumentations.Cutout(p=1.), 
#             ToTensorV2()],
#             p=1.)

# TTA6_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.ShiftScaleRotate(p=1.),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             ToTensorV2()],
#             p=1.)

# TTA7_deit = albumentations.Compose([
#             albumentations.CenterCrop(600, 600,p=1),
#             albumentations.Resize(deit_size,deit_size,cv2.INTER_AREA),
#             albumentations.Normalize(
#                 mean=[0.485, 0.456, 0.406], 
#                 std=[0.229, 0.224, 0.225], 
#                 max_pixel_value=255.0, 
#                 p=1.0
#             ),
#             albumentations.CoarseDropout(p=1.),
#             ToTensorV2()],
#             p=1.)

In [42]:
TTA0_deit = albumentations.Compose([
            albumentations.Resize(deit_size, deit_size,p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA1_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.HorizontalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.VerticalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_5_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.Transpose(p=1.),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA3_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1
            ),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA4_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=1.
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA5_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=1.), 
            ToTensorV2()],
            p=1.)

TTA6_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.ShiftScaleRotate(p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA7_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=1.),
            ToTensorV2()],
            p=1.)

In [43]:
TTA0 = albumentations.Compose([
            albumentations.Resize(sz, sz,p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA1 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.HorizontalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.VerticalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_5 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=1.),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA3 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1
            ),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA4 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=1.
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA5 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=1.), 
            ToTensorV2()],
            p=1.)

TTA6 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.ShiftScaleRotate(p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA7 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=1.),
            ToTensorV2()],
            p=1.)

In [44]:
infer_augs_effnet = albumentations.Compose([
            albumentations.RandomResizedCrop(sz_efffnetb4, sz_efffnetb4),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)


infer_augs_resnext = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)

### PL module

In [45]:
def freeze_until(net: Any, param_name: str = None) -> None:
    """
    Freeze net until param_name


    Args:
        net:
        param_name:

    """
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

class BasicEncoder(nn.Module):
    def __init__(
        self,
        arch: str = 'resnet18',
        source: str = 'torchvision',
        pretrained: str = None,
        n_layers: int = -1,
        freeze: bool = False,
        to_one_channel: bool = False,
        freeze_until_layer: str = None,
    ) -> None:
        """
        Initialize Encoder.

        Args:
            num_classes: the number of target classes, the size of the last layer's output
            arch: the name of the architecture form pretrainedmodels
            pretrained: the mode for pretrained model from pretrainedmodels
            n_layers: number of layers to keep
            freeze: to freeze model
            freeze_until: freeze until this layer. If None, then freeze all layers
        """
        super().__init__()
        if source == 'timm':
            net = timm.create_model(arch, pretrained=pretrained)
            self.output_dimension = list(net.children())[-1].in_features
        else:
            print("Not implemented")
            
        if freeze:
            freeze_until(net, freeze_until_layer)
        
        layers = list(net.children())[:n_layers]    
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        output = self.layers(x)

        return output

In [46]:
class BasicDecoder(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Linear(output_dimension , n_classes)

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [47]:
class BasicDecoder_newhead(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output


In [48]:
class Net_B4_newhead(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [49]:
class Net_B3(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b3_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [50]:
class Net_B4(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [51]:
class Net_B5(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b5_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [52]:
class BasicDecoder_seresnext(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        #self.fc = nn.Linear(output_dimension , n_classes)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [53]:
class seResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='seresnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_seresnext(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [54]:
class ResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [55]:
class ResnextNet50d(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50d_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [56]:
class resnest50d(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnest50d_1s4x24d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [57]:
class DEIT(nn.Module):
    def __init__(self) -> None:
        
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_384', pretrained=True)
        output_dimension = list(self.encoder.children())[-1].in_features

        self.encoder.head = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
        self.loss = nn.CrossEntropyLoss()
    def forward(self, x, targets):
        logits = self.encoder(x)
        loss = self.loss(logits, targets).view(1)
        return logits, loss


In [58]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
        self.metric = pl.metrics.Accuracy()
        self.learning_rate = 1e-4

    def forward(self, x, targets, *args, **kwargs):
        return self.model(x, targets)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

        return (
            [optimizer],
            [{'scheduler': scheduler, 'interval': 'epoch', 'monitor': 'valid_loss'}],
        )

    def training_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'train_loss': loss, f'train_accuracy': score}
        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'train_accuracy': score,
        }

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)
        
        logs = {'train_loss': avg_loss, 'train_accuracy': score}
        return {'log': logs, 'progress_bar': logs}

    def validation_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'valid_loss': loss, f'valid_accuracy': score}

        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'valid_accuracy': score,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)

        # score = torch.tensor(1.0, device=self.device)
        logs = {'valid_loss': avg_loss, f'valid_accuracy': score, 'accuracy': score}
        return {'valid_loss': avg_loss, 'log': logs, 'progress_bar': logs}

In [59]:
labels.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [60]:
# path = "../input/cassava-leaf-disease-classification/"
# sub = pd.read_csv(f'{path}/sample_submission.csv')
# sub.head()

In [61]:
def getloader(df,path,bs=8,fold=0,tta=0,deit=False):
    
    if not deit:
        if tta == 0:
            augs = TTA0
        if tta == 1:
            augs = TTA1
        if tta == 2:
            augs = TTA2
        if tta == 3:
            augs = TTA2_5
        if tta == 4:
            augs = TTA3
        if tta == 5:
            augs = TTA4
        if tta == 6:
            augs = TTA5
        if tta == 7:
            augs = TTA6
        if tta == 8:
            augs = TTA7
    else:
        if tta == 0:
            augs = TTA0_deit
        if tta == 1:
            augs = TTA1_deit
        if tta == 2:
            augs = TTA2_deit
        if tta == 3:
            augs = TTA2_5_deit
        if tta == 4:
            augs = TTA3_deit
        if tta == 5:
            augs = TTA4_deit
        if tta == 6:
            augs = TTA5_deit
        if tta == 7:
            augs = TTA6_deit
        if tta == 8:
            augs = TTA7_deit
    
    print(f"Using TTA: {tta} and DEIT:{deit}")
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_indexes, valid_indexes = list(folds.split(df, df['label']))[fold]

    train_df = df.iloc[train_indexes]
    valid_df = df.iloc[valid_indexes]


    train_dataset = ImageClassificationDataset(image_names=train_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=train_df['label'].values,
                                                    img_path=path,
                                                    mode='train',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    valid_dataset = ImageClassificationDataset(image_names=valid_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=valid_df['label'].values,
                                                    img_path=path,
                                                    mode='valid',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=bs,num_workers=0,shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=bs,num_workers=0,shuffle=False)

    return train_loader,valid_loader,valid_indexes

In [62]:
# models = os.listdir('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss/')
# modelsPath = [os.path.join('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss',i) for i in models]
# modelsPath

In [63]:
from tqdm import tqdm
def getPreds(test_loader,lit_model,logitsOut=True):
    predictions = []
    
    for batch in tqdm(test_loader):
        image = batch['image'].to('cuda')
        target = batch['target'].to('cuda')
        with torch.no_grad():
            outputs = lit_model.model(image, target)[0]
            _outputs = outputs.softmax(1).detach().cpu().numpy()
            preds = outputs.argmax(1).detach().cpu().numpy()
            if not logitsOut:
                predictions.append(preds)
            else:
                #predictions[range(len(_outputs))] += _outputs
                predictions.append(_outputs)
    return np.array(predictions)

In [64]:
TTA = 3 

### Preds for DEIT

In [65]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_4.ckpt'}
             


In [66]:
preds_DEIT = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = DEIT()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta,deit=True)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_DEIT[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


Using cache found in C:\Users\Rajneesh Tiwari/.cache\torch\hub\facebookresearch_deit_main
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:True


100%|██████████| 134/134 [01:05<00:00,  2.05it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:True


...

100%|██████████| 134/134 [01:49<00:00,  1.23it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:True


100%|██████████| 134/134 [01:14<00:00,  1.80it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:True


100%|██████████| 134/134 [01:15<00:00,  1.79it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [67]:
preds_mean_DEIT = preds_DEIT.copy()
preds_mean_DEIT[:,0]= preds_DEIT[:,0]/(1+np.sqrt(0.05))
preds_mean_DEIT[:,1]= preds_DEIT[:,1]/(1+np.sqrt(0.102))
preds_mean_DEIT[:,2]= preds_DEIT[:,2]/(1+np.sqrt(0.111))
preds_mean_DEIT[:,3]= preds_DEIT[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_DEIT[:,4]= preds_DEIT[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_DEIT,1)

In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_DEIT,1))

(0.8963873440201897, 0.8959667243071459)

In [69]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

403

### Preds for effnet B3

In [70]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'}
             


In [71]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [72]:
preds_effnet_b3 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B3()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b3[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.57it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:25<00:00,  1.56it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


100%|██████████| 134/134 [01:26<00:00,  1.56it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:False


100%|██████████| 134/134 [01:26<00:00,  1.55it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [73]:
preds_mean_effnetb3 = preds_effnet_b3.copy()
preds_mean_effnetb3[:,0]= preds_effnet_b3[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb3[:,1]= preds_effnet_b3[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb3[:,2]= preds_effnet_b3[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb3[:,3]= preds_effnet_b3[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb3[:,4]= preds_effnet_b3[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb3,1)

In [74]:

    
    
    
    
    
    
    from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b3,1))

(0.895546104594102, 0.896060195354489)

In [75]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

307

### Preds for seresnext

In [76]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [77]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [78]:
preds_seresnext = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):
    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = seResnextNet()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=128,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_seresnext[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 34/34 [01:55<00:00,  3.41s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 34/34 [01:40<00:00,  2.95s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


100%|██████████| 34/34 [01:40<00:00,  2.95s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:False


100%|██████████| 34/34 [01:40<00:00,  2.96s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [79]:
preds_mean_seresnext = preds_seresnext.copy()
preds_mean_seresnext[:,0]= preds_mean_seresnext[:,0]/(1+np.sqrt(0.05))
preds_mean_seresnext[:,1]= preds_mean_seresnext[:,1]/(1+np.sqrt(0.102))
preds_mean_seresnext[:,2]= preds_mean_seresnext[:,2]/(1+np.sqrt(0.111))
preds_mean_seresnext[:,3]= preds_mean_seresnext[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_seresnext[:,4]= preds_mean_seresnext[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_seresnext,1)

In [80]:
# classOut = np.argmax(preds_seresnext,1)
# classOut.shape

In [81]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_seresnext,1))

(0.8987241202037669, 0.8974155255409637)

In [82]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

298

In [83]:
torch.cuda.empty_cache()

In [84]:
gc.collect()

40

### Preds for Effnet b4 Taylor Loss - 2 SWA

In [85]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_4.ckpt'
             }

In [86]:
preds_effnet_b4_taylor = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4_taylor[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:22<00:00,  1.62it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:22<00:00,  1.63it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


100%|██████████| 134/134 [01:24<00:00,  1.58it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:False


100%|██████████| 134/134 [01:23<00:00,  1.60it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [87]:
preds_mean_effnetb4_taylor = preds_effnet_b4_taylor.copy()
preds_mean_effnetb4_taylor[:,0]= preds_effnet_b4_taylor[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4_taylor[:,1]= preds_effnet_b4_taylor[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4_taylor[:,2]= preds_effnet_b4_taylor[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4_taylor[:,3]= preds_effnet_b4_taylor[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4_taylor[:,4]= preds_effnet_b4_taylor[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4_taylor,1)

In [88]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_taylor,1))

(0.8967144926858905, 0.8974622610646352)

In [89]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for Effnet b4 Taylor Loss - 3 SWA

In [90]:
# modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
#               1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
#               2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
#               3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
#               4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
#              }


In [91]:
# preds_effnet_b4_taylor_3SWA = np.zeros((len(labels),5))

# train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
# TTA = TTA
# for f in range(5):    
#     softmaxOut = 0.
#     print(f"Prediction for fold:{f}")
#     modelPath = modelsPath[f]
#     model = Net_B4_newhead()
#     lit_model = LitCassava(model)
#     checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
#     lit_model.load_state_dict(checkpoint)      
# #    lit_model.load_state_dict(checkpoint['state_dict'])  
#     lit_model.model.eval()
#     lit_model.model.cuda()
#     for tta in range(TTA):
#         _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
#         predictions = []
#         softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
#     preds_effnet_b4_taylor_3SWA[valid_ix,:] = np.vstack(softmaxOut)

In [92]:
# preds_mean_effnetb4_taylor_3SWA = preds_effnet_b4_taylor_3SWA.copy()
# preds_mean_effnetb4_taylor_3SWA[:,0]= preds_effnet_b4_taylor_3SWA[:,0]/(1+np.sqrt(0.05))
# preds_mean_effnetb4_taylor_3SWA[:,1]= preds_effnet_b4_taylor_3SWA[:,1]/(1+np.sqrt(0.102))
# preds_mean_effnetb4_taylor_3SWA[:,2]= preds_effnet_b4_taylor_3SWA[:,2]/(1+np.sqrt(0.111))
# preds_mean_effnetb4_taylor_3SWA[:,3]= preds_effnet_b4_taylor_3SWA[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
# preds_mean_effnetb4_taylor_3SWA[:,4]= preds_effnet_b4_taylor_3SWA[:,4]/(1+np.sqrt(0.12))

# classOut = np.argmax(preds_mean_effnetb4_taylor_3SWA,1)

In [93]:
# from sklearn.metrics import accuracy_score
# accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_taylor_3SWA,1))

In [94]:
# del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for B4 focal cosine loss

In [95]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [96]:
preds_effnet_b4 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:21<00:00,  1.64it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:21<00:00,  1.64it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


100%|██████████| 134/134 [01:22<00:00,  1.63it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:False


100%|██████████| 134/134 [01:22<00:00,  1.62it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [97]:
preds_mean_effnetb4 = preds_effnet_b4.copy()
preds_mean_effnetb4[:,0]= preds_effnet_b4[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4[:,1]= preds_effnet_b4[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4[:,2]= preds_effnet_b4[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4[:,3]= preds_effnet_b4[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4[:,4]= preds_effnet_b4[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4,1)

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4,1))

(0.8977426742066645, 0.8978361452540076)

In [99]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble b4 with deit

In [100]:
res_ = pd.DataFrame()
res_['b4_contrib'] = 0
#res['b5_contrib'] = 0
res_['deit_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    deit_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4+deit_contrib*preds_DEIT)
    res_.loc[count,'b4_contrib'] = i
    res_.loc[count,'deit_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [101]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,Accuracy
53,0.53,0.47,0.902276
49,0.49,0.51,0.902136
47,0.47,0.53,0.902136
52,0.52,0.48,0.902042
48,0.48,0.52,0.902042
...,...,...,...
3,0.03,0.97,0.896714
5,0.05,0.95,0.896621
2,0.02,0.98,0.896528
1,0.01,0.99,0.896200


### Ensemble 

In [102]:
res_ = pd.DataFrame()
res_['b4_contrib'] = 0
#res['b5_contrib'] = 0
res_['seresnext_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    seresnext_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext)
    res_.loc[count,'b4_contrib'] = i
    res_.loc[count,'seresnext_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [103]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,seresnext_contrib,Accuracy
24,0.24,0.76,0.901341
22,0.22,0.78,0.901341
18,0.18,0.82,0.901295
23,0.23,0.77,0.901248
25,0.25,0.75,0.901248
...,...,...,...
96,0.96,0.04,0.897883
97,0.97,0.03,0.897836
98,0.98,0.02,0.897789
99,0.99,0.01,0.897696


### Ensemble deit with b4 taylor loss

In [104]:
res_ = pd.DataFrame()
res_['b4_taylor_contrib'] = 0
#res['b5_contrib'] = 0
res_['deit_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    deit_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4_taylor+deit_contrib*preds_DEIT)
    res_.loc[count,'b4_taylor_contrib'] = i
    res_.loc[count,'deit_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [105]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_taylor_contrib,deit_contrib,Accuracy
44,0.44,0.56,0.901295
43,0.43,0.57,0.901201
40,0.40,0.60,0.901201
41,0.41,0.59,0.901154
39,0.39,0.61,0.901154
...,...,...,...
4,0.04,0.96,0.896574
3,0.03,0.97,0.896434
2,0.02,0.98,0.896387
1,0.01,0.99,0.896200


### Ensemble seresnext with b4 taylor loss

In [106]:
res_ = pd.DataFrame()
res_['b4_taylor_contrib'] = 0
#res['b5_contrib'] = 0
res_['seresnext_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    seresnext_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4_taylor+seresnext_contrib*preds_seresnext)
    res_.loc[count,'b4_taylor_contrib'] = i
    res_.loc[count,'seresnext_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [107]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_taylor_contrib,seresnext_contrib,Accuracy
24,0.24,0.76,0.901528
23,0.23,0.77,0.901295
22,0.22,0.78,0.901295
21,0.21,0.79,0.901295
25,0.25,0.75,0.901061
...,...,...,...
97,0.97,0.03,0.897462
99,0.99,0.01,0.897462
96,0.96,0.04,0.897462
0,0.00,1.00,0.897416


### Ensemble deit+seresnext+b4 focal cosine

In [108]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['deit_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        deit_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+deit_contrib*preds_mean_DEIT)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'deit_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [109]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1823,0.20,0.13,0.67,0.903117,0.902510
1741,0.19,0.12,0.69,0.903071,0.902276
1903,0.21,0.13,0.66,0.903024,0.902229
1904,0.21,0.14,0.65,0.902977,0.902136
1660,0.18,0.13,0.69,0.902977,0.902884
...,...,...,...,...,...
94,0.00,0.94,0.06,0.896200,0.896387
93,0.00,0.93,0.07,0.896154,0.896481
97,0.00,0.97,0.03,0.896107,0.896341
98,0.00,0.98,0.02,0.896107,0.896341


In [110]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
3823,0.50,0.45,0.05,0.902136,0.903164
3772,0.49,0.45,0.06,0.902089,0.903117
3719,0.48,0.44,0.08,0.902136,0.903117
2591,0.30,0.26,0.44,0.902650,0.903117
3874,0.51,0.46,0.03,0.902089,0.903117
...,...,...,...,...,...
95,0.00,0.95,0.05,0.896247,0.896387
99,0.00,0.99,0.01,0.896013,0.896341
98,0.00,0.98,0.02,0.896107,0.896341
97,0.00,0.97,0.03,0.896107,0.896341


### Preds for B5

In [111]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [112]:
preds_effnet_b5 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B5()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b5[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:33<00:00,  1.43it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:33<00:00,  1.43it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


100%|██████████| 134/134 [01:34<00:00,  1.42it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.41it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [113]:
preds_mean_effnetb5 = preds_effnet_b5.copy()
preds_mean_effnetb5[:,0]= preds_effnet_b5[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb5[:,1]= preds_effnet_b5[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb5[:,2]= preds_effnet_b5[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb5[:,3]= preds_effnet_b5[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb5[:,4]= preds_effnet_b5[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb5,1)

In [114]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b5,1))

(0.8961069308781605, 0.8973220544936206)

In [115]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble

In [116]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['b5_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        b5_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'b5_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [117]:
pd.set_option('display.max_rows',1000)

In [118]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
2506,0.29,0.12,0.59,0.902697,0.901949
2379,0.27,0.30,0.43,0.902650,0.901949
2304,0.26,0.29,0.45,0.902650,0.901949
2385,0.27,0.36,0.37,0.902650,0.902042
2458,0.28,0.36,0.36,0.902603,0.902183
...,...,...,...,...,...
98,0.00,0.98,0.02,0.897602,0.896294
197,0.01,0.97,0.02,0.897509,0.896341
196,0.01,0.96,0.03,0.897509,0.896574
99,0.00,0.99,0.01,0.897462,0.896154


In [119]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1074,0.11,0.29,0.60,0.902369,0.903211
891,0.09,0.27,0.64,0.902463,0.903164
892,0.09,0.28,0.63,0.902323,0.903164
890,0.09,0.26,0.65,0.902369,0.903117
893,0.09,0.29,0.62,0.902510,0.903117
...,...,...,...,...,...
197,0.01,0.97,0.02,0.897509,0.896341
198,0.01,0.98,0.01,0.897696,0.896294
98,0.00,0.98,0.02,0.897602,0.896294
97,0.00,0.97,0.03,0.897696,0.896294


### resnest50d_1s4x24d

In [120]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [121]:
preds_resnest50d = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = resnest50d()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_resnest50d[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.58it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:25<00:00,  1.57it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.56it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:False


100%|██████████| 134/134 [01:26<00:00,  1.55it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [122]:
preds_mean_resnest50d = preds_resnest50d.copy()
preds_mean_resnest50d[:,0]= preds_resnest50d[:,0]/(1+np.sqrt(0.05))
preds_mean_resnest50d[:,1]= preds_resnest50d[:,1]/(1+np.sqrt(0.102))
preds_mean_resnest50d[:,2]= preds_resnest50d[:,2]/(1+np.sqrt(0.111))
preds_mean_resnest50d[:,3]= preds_resnest50d[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_resnest50d[:,4]= preds_resnest50d[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_resnest50d,1)

In [123]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_resnest50d,1))

(0.8937234191709118, 0.8968079637332337)

### Resnext50D

In [124]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [125]:
preds_resnext50d = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = ResnextNet50d()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
#     lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_resnext50d[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.40it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:34<00:00,  1.42it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.40it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2 and DEIT:False


100%|██████████| 134/134 [01:38<00:00,  1.36it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [126]:
preds_mean_resnext50d = preds_resnext50d.copy()
preds_mean_resnext50d[:,0]= preds_resnext50d[:,0]/(1+np.sqrt(0.05))
preds_mean_resnext50d[:,1]= preds_resnext50d[:,1]/(1+np.sqrt(0.102))
preds_mean_resnext50d[:,2]= preds_resnext50d[:,2]/(1+np.sqrt(0.111))
preds_mean_resnext50d[:,3]= preds_resnext50d[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_resnext50d[:,4]= preds_resnext50d[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_resnext50d,1)

In [127]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_resnest50d,1))

(0.8930691218395103, 0.8968079637332337)

In [134]:
res_all4 = pd.DataFrame()
res_all4['b4_contrib'] = 0
res_all4['b5_contrib'] = 0
res_all4['seresnext_contrib'] = 0
res_all4['resnest_contrib'] = 0
res_all4['DEIT_contrib'] = 0

res_all4 ['Accuracy'] = 0
res_all4 ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.05):
    for j in np.arange(0,1-i,0.05):
        for k in np.arange(0,1-i-j,0.05):
            for l in np.arange(0,1-i-j-k,0.05):
                b4_contrib = i
                b5_contrib = j
                seresnext_contrib = k
                resnest_contrib = l = 0
                deit_contrib = 1-i-j-k-l
                print(b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib+deit_contrib)
                #assert b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib==1.
                #print(i,j,k,1-(i+j+k))
                ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5+resnest_contrib*preds_mean_resnest50d+deit_contrib*preds_mean_DEIT)
                ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5+resnest_contrib*preds_resnest50d+deit_contrib*preds_DEIT)
                res_all4.loc[count,'b4_contrib'] = i
                res_all4.loc[count,'seresnext_contrib'] = j
                res_all4.loc[count,'b5_contrib'] = k
                res_all4.loc[count,'resnest_contrib'] = l
                res_all4.loc[count,'DEIT_contrib'] = 1-i-j-k-l
                res_all4.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
                res_all4.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
                count += 1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.999999

0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999

0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999998
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1

In [135]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,resnest_contrib,DEIT_contrib,Accuracy,Adj_Accuracy
5668,0.2,0.25,0.3,0.0,0.25,0.904005,0.903631
5667,0.2,0.25,0.3,0.0,0.25,0.904005,0.903631
5666,0.2,0.25,0.3,0.0,0.25,0.904005,0.903631
5665,0.2,0.25,0.3,0.0,0.25,0.904005,0.903631
5664,0.2,0.25,0.3,0.0,0.25,0.904005,0.903631
...,...,...,...,...,...,...,...
16,0.0,0.00,0.0,0.0,1.00,0.895967,0.896387
17,0.0,0.00,0.0,0.0,1.00,0.895967,0.896387
18,0.0,0.00,0.0,0.0,1.00,0.895967,0.896387
19,0.0,0.00,0.0,0.0,1.00,0.895967,0.896387


### Ensemble

In [129]:
res_all4 = pd.DataFrame()
res_all4['b4_contrib'] = 0
res_all4['b5_contrib'] = 0
res_all4['seresnext_contrib'] = 0
res_all4['resnest_contrib'] = 0
res_all4['DEIT_contrib'] = 0

res_all4 ['Accuracy'] = 0
res_all4 ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.05):
    for j in np.arange(0,1-i,0.05):
        for k in np.arange(0,1-i-j,0.05):
            for l in np.arange(0,1-i-j-k,0.05):
                b4_contrib = i
                b5_contrib = j
                seresnext_contrib = k
                resnest_contrib = l
                deit_contrib = 1-i-j-k-l
                print(b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib+deit_contrib)
                #assert b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib==1.
                #print(i,j,k,1-(i+j+k))
                ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5+resnest_contrib*preds_mean_resnest50d+deit_contrib*preds_mean_DEIT)
                ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5+resnest_contrib*preds_resnest50d+deit_contrib*preds_DEIT)
                res_all4.loc[count,'b4_contrib'] = i
                res_all4.loc[count,'seresnext_contrib'] = j
                res_all4.loc[count,'b5_contrib'] = k
                res_all4.loc[count,'resnest_contrib'] = l
                res_all4.loc[count,'DEIT_contrib'] = 1-i-j-k-l
                res_all4.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
                res_all4.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
                count += 1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0

1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999998
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999998
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
0

0.9999999999999999
0.9999999999999999
0.9999999999999998
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999998
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999998
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999998
0.9999999999999999
0.9999999999999998
0.9999999999999998
0.9999999999999998
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.999999999

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0000000000000002
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999998
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999998
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0

1.0000000000000002
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0000000000000002
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0000000000000002
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [130]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,resnest_contrib,DEIT_contrib,Accuracy,Adj_Accuracy
2016,0.05,0.35,0.10,0.40,0.10,0.904145,0.903304
839,0.00,0.45,0.20,0.25,0.10,0.904099,0.903538
3655,0.10,0.50,0.25,0.05,0.10,0.904052,0.902650
1788,0.05,0.10,0.05,0.70,0.10,0.904052,0.903164
3356,0.10,0.00,0.15,0.60,0.15,0.904052,0.903117
...,...,...,...,...,...,...,...
20,0.00,0.05,0.00,0.00,0.95,0.896247,0.896387
39,0.00,0.10,0.00,0.00,0.90,0.896247,0.896621
21,0.00,0.05,0.00,0.05,0.90,0.896200,0.896387
1,0.00,0.00,0.00,0.05,0.95,0.896013,0.896247


In [131]:
res_all4.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,resnest_contrib,DEIT_contrib,Accuracy,Adj_Accuracy
694,0.00,0.45,0.15,0.25,0.15,0.903351,0.904286
707,0.00,0.55,0.15,0.15,0.15,0.903164,0.904145
701,0.00,0.50,0.15,0.20,0.15,0.903538,0.904052
6279,0.25,0.10,0.20,0.15,0.30,0.903257,0.904005
712,0.00,0.60,0.15,0.10,0.15,0.903211,0.904005
...,...,...,...,...,...,...,...
20,0.00,0.05,0.00,0.00,0.95,0.896247,0.896387
21,0.00,0.05,0.00,0.05,0.90,0.896200,0.896387
3,0.00,0.00,0.00,0.15,0.85,0.896294,0.896294
1,0.00,0.00,0.00,0.05,0.95,0.896013,0.896247


In [ ]:
#preds_DEIT
#preds_mean_DEIT

In [132]:
res_all4 = pd.DataFrame()
res_all4['DEIT_contrib'] = 0
res_all4['b5_contrib'] = 0
res_all4['seresnext_contrib'] = 0
res_all4['resnest_contrib'] = 0

res_all4 ['Accuracy'] = 0
res_all4 ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.05):
    for j in np.arange(0,1-i,0.05):
        for k in np.arange(0,1-i-j,0.05):
            DEIT_contrib = i
            b5_contrib = j 
            seresnext_contrib = k = 0 
            resnest_contrib = 1-i-j-k
            print(DEIT_contrib+b5_contrib+seresnext_contrib+resnest_contrib)
            #assert b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib==1.
            #print(i,j,k,1-(i+j+k))
            ens_preds_adj = (DEIT_contrib*preds_mean_DEIT+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5+resnest_contrib*preds_mean_resnest50d)
            ens_preds = (DEIT_contrib*preds_DEIT+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5+resnest_contrib*preds_resnest50d)
            res_all4.loc[count,'DEIT_contrib'] = i
            res_all4.loc[count,'seresnext_contrib'] = j
            res_all4.loc[count,'b5_contrib'] = k
            res_all4.loc[count,'resnest_contrib'] = 1-i-j-k
            res_all4.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
            res_all4.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
            count += 1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.999999

In [133]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

,DEIT_contrib,b5_contrib,seresnext_contrib,resnest_contrib,Accuracy,Adj_Accuracy
469,0.10,0.0,0.2,0.70,0.903304,0.902229
466,0.10,0.0,0.2,0.70,0.903304,0.902229
467,0.10,0.0,0.2,0.70,0.903304,0.902229
468,0.10,0.0,0.2,0.70,0.903304,0.902229
470,0.10,0.0,0.2,0.70,0.903304,0.902229
...,...,...,...,...,...,...
1540,0.85,0.0,0.0,0.15,0.896294,0.896294
1541,0.85,0.0,0.0,0.15,0.896294,0.896294
1545,0.90,0.0,0.0,0.10,0.896294,0.896107
1546,0.90,0.0,0.0,0.10,0.896294,0.896107


In [ ]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_effnet_b4,1))

In [ ]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_effnet_b5,1))

In [ ]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_seresnext,1))

In [ ]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_resnest50d,1))

In [ ]:
np.corrcoef(np.max(preds_seresnext,1),np.max(preds_resnest50d,1))

In [ ]:
np.corrcoef(np.max(preds_effnet_b4_taylor,1),np.max(preds_effnet_b4,1))

In [ ]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(labels.label,np.argmax(preds_effnet_b4,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_mean_effnetb4,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_effnet_b4_taylor,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_mean_effnetb4_taylor,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_DEIT,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_effnet_b5,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_resnest50d,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_resnext50d,1),normalize='true')

In [ ]:
confusion_matrix(labels.label,np.argmax(preds_seresnext,1),normalize='true')

In [ ]:
b4_contrib = 0.18
seresnext_contrib = 0.61
b5_contrib = 0.21
_ = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

In [ ]:
deit_contrib = 0.15
seresnext_contrib = 0.25
resnest_contrib = 0.6
_ = (deit_contrib*preds_DEIT+seresnext_contrib*preds_seresnext+resnest_contrib*preds_resnest50d)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

In [ ]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['deit_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        deit_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+deit_contrib*preds_mean_DEIT)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'deit_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [ ]:
res.sort_values(by=['Accuracy'],ascending=False)

In [ ]:
labels.label.value_counts()

### Below looks good

In [ ]:
deit_contrib = 0.25
seresnext_contrib = 0.46
b4_contrib = 0.29
_ = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')